In Lake Formation enabled clusters, Spark SQL can only read from data managed by AWS Glue Data Catalog and cannot access data managed outside of AWS Glue or Lake Formation. Data from other sources can be accessed using non-Spark SQL operations if the IAM role for other AWS Services chosen during cluster deployment has policies in place allowing the cluster to access those data sources

Spark SQL can only read from Lake Formation tables

So let's see how may databases you have access to, one of them should be tpc as SAML authenticatd user has SELECT permission for two of the tables in tpc database.

In [5]:
spark.sql("show databases").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+
|databaseName|
+------------+
|     default|
|    sampledb|
|         tpc|
+------------+



As we gave SELECT permission to dl_tpc_web_page table under tpc database , let's get the count(*) of rows from the table .

In [2]:
spark.sql( "select count(*) from tpc.dl_tpc_web_page ").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|count(1)|
+--------+
|    3600|
+--------+



Let's select table for which user does not have SELECT permission and see what happens? You should receive Service: AWSGlue; Status Code: 400; Error Code: AccessDeniedException for the following 

In [3]:
spark.sql("SELECT * FROM tpc.dl_tpc_item limit 10").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

org.apache.spark.sql.AnalysisException: org.apache.hadoop.hive.ql.metadata.HiveException: Unable to fetch table dl_tpc_item. Insufficient Lake Formation permission(s) on dl_tpc_item (Service: AWSGlue; Status Code: 400; Error Code: AccessDeniedException; Request ID: 52cf5f90-df6f-4a08-9cb5-ccf701c06f8c);
  at org.apache.spark.sql.hive.HiveExternalCatalog.withClient(HiveExternalCatalog.scala:106)
  at org.apache.spark.sql.hive.HiveExternalCatalog.tableExists(HiveExternalCatalog.scala:824)
  at org.apache.spark.sql.catalyst.catalog.ExternalCatalogWithListener.tableExists(ExternalCatalogWithListener.scala:142)
  at org.apache.spark.sql.catalyst.catalog.SessionCatalog.tableExists(SessionCatalog.scala:420)
  at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveRelations$.isRunningDirectlyOnFiles(Analyzer.scala:764)
  at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveRelations$.resolveRelation(Analyzer.scala:697)
  at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveRelatio

For dl_tpc_customer table we only give SELECT permission at four columns COLUMN which includes ( c_first_sales_date_sk, c_first_name,c_last_name,c_first_shipto_date_sk ) following query should only show those columns for which user has access. 

In [4]:
spark.sql("select * from tpc.dl_tpc_customer limit 10").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+------------+-----------+----------------------+
|c_first_sales_date_sk|c_first_name|c_last_name|c_first_shipto_date_sk|
+---------------------+------------+-----------+----------------------+
|                 null|        null|       null|               2449597|
|              2452077|       Joyce|     Deaton|                  null|
|              2450637|       Ellis|        Dow|               2450667|
|              2452342|        null|       Cass|               2452372|
|                 null|        null|      Lange|                  null|
|                 null|        Rene| Mclaughlin|                  null|
|                 null|       Megan|      Sisco|                  null|
|                 null|        null|       Soto|                  null|
|                 null|      Stella|       null|               2449664|
|              2449021|      Wesley|     Harris|                  null|
+---------------------+------------+-----------+----------------